### 552 Final Project -- News Crawler

This script aims to crawl the first paragraph of all the articles of 'covid vaccine' in the new york times archive.

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
import time
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup

In [488]:
option = ChromeOptions()
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = Chrome(options = option, executable_path = './chromedriver')

In [550]:
url = 'https://www.nytimes.com/search?dropmab=true&endDate=20200706&query=covid&sections=Health%7Cnyt%3A%2F%2Fsection%2F9f943015-a899-5505-8730-6d30ed861520&sort=newest&types=article'
driver.get(url)

In [99]:
# The search result only contains 10 results per page
# To load the full page we need to click the button 'show more'
def load_full_page(driver):
    i = 1
    while True:
        try:
            show_more = driver.find_element_by_css_selector('[data-testid="search-show-more-button"]')
            y = show_more.location.get('y')
            click_show_more = show_more.click()
            time.sleep(2)

            show_more_new = driver.find_element_by_css_selector('[data-testid="search-show-more-button"]')
            y_new = show_more_new.location.get('y')

            if y != y_new:
                print(f'page {i} has been loaded')
                click_show_more_new = show_more_new.click()

            else:
                print('The page is fully loaded.')
                break

            i += 1
        except:
            break
            
    return driver

In [5]:
def get_page_link(driver):
    #locate data
    title_list = []
    link_list = []
    
    # Locate the content list
    contents = driver.find_elements_by_xpath("/html/body/div[1]/div[2]/main/div/div[2]/div[2]/ol/li")  
    for li in contents:
        try:
            link = li.find_element_by_css_selector('.css-e1lvw9 [href]')
            l = link.get_attribute('href')
            link_list.append(l)
            
            title = li.find_element_by_class_name('css-2fgx4k')
            ti = title.text
            title_list.append(ti)
            
            print(len(title_list),'links are retrived')
            
        except:
            continue
            
    return title_list, link_list

#### Load the full page

In [551]:
loaded_driver = load_full_page(driver)

page 1 has been loaded
page 2 has been loaded
page 3 has been loaded
page 4 has been loaded
page 5 has been loaded
page 6 has been loaded
page 7 has been loaded
page 8 has been loaded
page 9 has been loaded
page 10 has been loaded
page 11 has been loaded
page 12 has been loaded
page 13 has been loaded
page 14 has been loaded
page 15 has been loaded
page 16 has been loaded
page 17 has been loaded
page 18 has been loaded
page 19 has been loaded
page 20 has been loaded
page 21 has been loaded
page 22 has been loaded
page 23 has been loaded
page 24 has been loaded
page 25 has been loaded
page 26 has been loaded
page 27 has been loaded
page 28 has been loaded
page 29 has been loaded
page 30 has been loaded
page 31 has been loaded
page 32 has been loaded
page 33 has been loaded
page 34 has been loaded
page 35 has been loaded
page 36 has been loaded
page 37 has been loaded


#### Retrive all the links

In [552]:
t_list, l_list = get_page_link(loaded_driver)

1 links are retrived
2 links are retrived
3 links are retrived
4 links are retrived
5 links are retrived
6 links are retrived
7 links are retrived
8 links are retrived
9 links are retrived
10 links are retrived
11 links are retrived
12 links are retrived
13 links are retrived
14 links are retrived
15 links are retrived
16 links are retrived
17 links are retrived
18 links are retrived
19 links are retrived
20 links are retrived
21 links are retrived
22 links are retrived
23 links are retrived
24 links are retrived
25 links are retrived
26 links are retrived
27 links are retrived
28 links are retrived
29 links are retrived
30 links are retrived
31 links are retrived
32 links are retrived
33 links are retrived
34 links are retrived
35 links are retrived
36 links are retrived
37 links are retrived
38 links are retrived
39 links are retrived
40 links are retrived
41 links are retrived
42 links are retrived
43 links are retrived
44 links are retrived
45 links are retrived
46 links are retriv

363 links are retrived
364 links are retrived
365 links are retrived
366 links are retrived
367 links are retrived
368 links are retrived
369 links are retrived
370 links are retrived
371 links are retrived
372 links are retrived
373 links are retrived
374 links are retrived
375 links are retrived
376 links are retrived
377 links are retrived
378 links are retrived
379 links are retrived
380 links are retrived
381 links are retrived
382 links are retrived
383 links are retrived
384 links are retrived
385 links are retrived
386 links are retrived
387 links are retrived
388 links are retrived
389 links are retrived
390 links are retrived
391 links are retrived
392 links are retrived
393 links are retrived
394 links are retrived
395 links are retrived
396 links are retrived
397 links are retrived
398 links are retrived
399 links are retrived
400 links are retrived
401 links are retrived
402 links are retrived
403 links are retrived
404 links are retrived
405 links are retrived
406 links a

723 links are retrived
724 links are retrived
725 links are retrived
726 links are retrived
727 links are retrived
728 links are retrived
729 links are retrived
730 links are retrived
731 links are retrived
732 links are retrived
733 links are retrived
734 links are retrived
735 links are retrived
736 links are retrived
737 links are retrived
738 links are retrived
739 links are retrived
740 links are retrived
741 links are retrived
742 links are retrived
743 links are retrived
744 links are retrived


In [553]:
print(len(l_list))

744


In [554]:
linklist = pd.DataFrame({'title':t_list, 'links':l_list})

In [555]:
linklist.to_csv('data/news_links_16.csv', index = False)

#### Retrive the first paragraph of each article

In [556]:
# Get all the links
links = pd.read_csv('data/news_links_16.csv')

In [557]:
links.head()

,title,links
0,"In Nick Cordero’s Death, a Reminder of Covid-1...",https://www.nytimes.com/2020/07/06/health/coro...
1,Airborne Coronavirus: What You Should Do Now,https://www.nytimes.com/2020/07/06/health/coro...
2,A New Generation of Fast Coronavirus Tests Is ...,https://www.nytimes.com/2020/07/06/health/fast...
3,239 Experts With One Big Claim: The Coronaviru...,https://www.nytimes.com/2020/07/04/health/239-...
4,DNA Inherited From Neanderthals May Increase R...,https://www.nytimes.com/2020/07/04/health/coro...


In [558]:
# Function that retrive the first paragraph of the given link
def first_p(link):
    p = requests.get(link)
    soup = BeautifulSoup(p.content, 'html.parser')
    try:
        fp = soup.find('p', {"class" : "css-axufdj evys1bk0"}).text
        try:
            fps = fp.split('—')[1].strip()
            return fps
        except:
            return fp
    except:
        return ''

In [559]:
# Retrive the first paragraph
fp_list = []

for i in range(len(links['links'])):
    l = links['links'][i]
    fp_list.append(first_p(l))
    print(i, 'pages are retrived')

0 pages are retrived
1 pages are retrived
2 pages are retrived
3 pages are retrived
4 pages are retrived
5 pages are retrived
6 pages are retrived
7 pages are retrived
8 pages are retrived
9 pages are retrived
10 pages are retrived
11 pages are retrived
12 pages are retrived
13 pages are retrived
14 pages are retrived
15 pages are retrived
16 pages are retrived
17 pages are retrived
18 pages are retrived
19 pages are retrived
20 pages are retrived
21 pages are retrived
22 pages are retrived
23 pages are retrived
24 pages are retrived
25 pages are retrived
26 pages are retrived
27 pages are retrived
28 pages are retrived
29 pages are retrived
30 pages are retrived
31 pages are retrived
32 pages are retrived
33 pages are retrived
34 pages are retrived
35 pages are retrived
36 pages are retrived
37 pages are retrived
38 pages are retrived
39 pages are retrived
40 pages are retrived
41 pages are retrived
42 pages are retrived
43 pages are retrived
44 pages are retrived
45 pages are retrive

362 pages are retrived
363 pages are retrived
364 pages are retrived
365 pages are retrived
366 pages are retrived
367 pages are retrived
368 pages are retrived
369 pages are retrived
370 pages are retrived
371 pages are retrived
372 pages are retrived
373 pages are retrived
374 pages are retrived
375 pages are retrived
376 pages are retrived
377 pages are retrived
378 pages are retrived
379 pages are retrived
380 pages are retrived
381 pages are retrived
382 pages are retrived
383 pages are retrived
384 pages are retrived
385 pages are retrived
386 pages are retrived
387 pages are retrived
388 pages are retrived
389 pages are retrived
390 pages are retrived
391 pages are retrived
392 pages are retrived
393 pages are retrived
394 pages are retrived
395 pages are retrived
396 pages are retrived
397 pages are retrived
398 pages are retrived
399 pages are retrived
400 pages are retrived
401 pages are retrived
402 pages are retrived
403 pages are retrived
404 pages are retrived
405 pages a

720 pages are retrived
721 pages are retrived
722 pages are retrived
723 pages are retrived
724 pages are retrived
725 pages are retrived
726 pages are retrived
727 pages are retrived
728 pages are retrived
729 pages are retrived
730 pages are retrived
731 pages are retrived
732 pages are retrived
733 pages are retrived
734 pages are retrived
735 pages are retrived
736 pages are retrived
737 pages are retrived
738 pages are retrived
739 pages are retrived
740 pages are retrived
741 pages are retrived
742 pages are retrived
743 pages are retrived


In [560]:
links['first_paragraph'] = fp_list

In [561]:
links.to_csv('data/news_links_16.csv', index = False)

#### Data cleaning

In [562]:
contents = pd.read_csv('data/news_links_16.csv')

In [563]:
# Drop those with short sentences.
flt = (contents['first_paragraph'].str.len()>100)
filtered_contents = contents.loc[flt]

In [564]:
filtered_contents.shape

(636, 3)

In [565]:
# Drop the duplicates
filtered_contents = filtered_contents.drop_duplicates(subset=['first_paragraph'])

In [566]:
filtered_contents.shape

(326, 3)

In [567]:
# Drop rows that do not end up with '.'
filtered_contents = filtered_contents.loc[filtered_contents['first_paragraph'].str.endswith('.')]

In [547]:
filtered_contents.shape

(465, 3)

In [568]:
filtered_contents.to_csv('data/news_content_16.csv', index=False)

In [569]:
# Output the training set
filtered_contents['first_paragraph'].to_csv('data/news_train_16.csv', index=False)

#### Combine files

In [570]:
for i in range (7,17):
    exec('df_links{} = pd.read_csv("data/news_links_{}.csv")'.format(i, i))
    exec('df_content{} = pd.read_csv("data/news_content_{}.csv")'.format(i, i))
    exec('df_train{} = pd.read_csv("data/news_train_{}.csv")'.format(i, i))

In [571]:
# Concat all the crawled file together
df_content = pd.concat([df_content1, df_content2, df_content3, df_content4, df_content5, df_content6,
                       df_content7, df_content8, df_content9, df_content10, df_content11, df_content12, 
                       df_content13, df_content14, df_content15, df_content16])

df_train = pd.concat([df_train1, df_train2, df_train3, df_train4, df_train5, df_train6,
                      df_train7, df_train8, df_train9, df_train10, df_train11, df_train12,
                     df_train13, df_train14, df_train15, df_train16])

In [572]:
#df_links.to_csv('news_links_all.csv', index=False)
df_content.to_csv('news_contents_all.csv', index=False)
df_train.to_csv('news_train_all.csv', index=False)